<h1>Creazione ontologia</h1>

Lo scopo principale del programma è quello di creare l'ontologia necessaria.
Per l'ontologia delle coordinate, utilizziamo Schema.org.

Librerie utilizzate:


In [8]:
import urllib
import csv
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, FOAF, OWL, XSD, DC, DCTERMS
import requests
import re
from geopy.distance import geodesic


Qui vengono definiti i <b>Namespace</b> utilizzati, ossia SchoolRank e Schema:

In [9]:
#skr = schoolrank
g = Graph()
skr = Namespace("http://www.schoolrank.org/ontology/")
schema = Namespace('https://schema.org/')
base_uri = "http://www.schoolrank.org/resource/"
g.bind("skr", skr)
g.bind('schema', schema)

Abbiamo realizzato l'ontologia sia per le fermate dell'autobus sia per le scuole e i relativi plessi: utilizziamo delle funzioni che "trasformano in URI" il nome della scuola (e del plesso), l'indirizzo e il nome della fermata:

<i>Nelle URI non sono ammessi spazi: vengono infatti sostituiti da degli underscore e vengono effettuate altre sostituzioni:
</i>

In [10]:
#Fermata
def urify_fermata(nomefermata):
    replace_dict = {
        ' - ':' ',
        " ": "_",
        "'": '_',
        ".": '',
        '"': '',
        '__':'_'
    }
    regex_dict = {

    }
    # replace
    for key, values in replace_dict.items():
        nomefermata = nomefermata.replace(key, values)
    # regex
    for key in regex_dict:
        rx = re.compile(key)
        nomefermata = rx.sub(regex_dict[key], nomefermata)
    return nomefermata


In [11]:
#Indirizzo scuola
def urify_indirizzo(indirizzo):
    replace_dict = {
        " ":"_",
        ".":'_',
        "'":"_",
        '__':'_'
    }
    regex_dict ={
        r'[_]*$' : '',
    }
    #replace
    for key, values in replace_dict.items():
        indirizzo = indirizzo.replace(key, values)
    #regex
    for key in regex_dict:
        rx = re.compile(key)
        indirizzo = rx.sub(regex_dict[key], indirizzo)
    return indirizzo


<i>Abbiamo eliminato molte abbreviazioni:</i>

In [12]:
#Nome scuola
def urify_scuola(nomescuola):
    replace_dict ={
        "I. FLORIO - S. LORENZO":"IGNAZIO FLORIO SAN LORENZO",
        "E. ARCULEO":"ETTORE ARCULEO",
        "F. SAVERIO CAVALLARI":"FRANCESCO SAVERIO CAVALLARI",
        "A. DE GASPERI":"ALCIDE DE GASPERI",
        "E. DE AMICIS":"EDMONDO DE AMICIS",
        "A. GABELLI":"ARISTIDE GABELLI",
        "N. GARZILLI":"NICOLO GARZILLI",
        "E. SALGARI":"EMILIO SALGARI",
        "F. ORESTANO":"FRANCESCO ORESTANO",
        "C. MANERI":"CARMELO MANERI",
        "M.TERESA DI CALCUTTA":"MADRE TERESA DI CALCUTTA",
        "G.A.COLOZZA":"GIOVANNI ANTONIO COLOZZA",
        "G. DI VITTORIO":"GIUSEPPE DI VITTORIO",
        "TOMASI DI L.":"TOMASI DI LAMPEDUSA",
        "G. E. NUCCIO":"GIUSEPPE ERNESTO NUCCIO",
        "L.DA VINCI /G.CARDUCCI":"LEONARDO DA VINCI GIOSUE CARDUCCI",
        "M.RAPISARDI":"MARIO RAPISARDI",
        "R. SANZIO":"RAFFAELLO SANZIO",
        "E. MEDI":"ENRICO MEDI",
        "A. VOLTA":"ALESSANDRO VOLTA",
        "E ASCIONE":"ERNESTO ASCIONE",
        "L.EINAUDI":"LUIGI EINAUDI",
        "CESAREO G.A.":"CESAREO GIOVANNI ALFREDO",
        "V.E. ORLANDO":"VITTORIO EMANUELE ORLANDO",
        "D. DOLCI":"DANILO DOLCI",
        "L.PIRANDELLO/B. ULIVIA":"LUIGI PIRANDELLO BORGO ULIVIA",
        "S.D'ACQUISTO BAGHERIA":"SALVO D' ACQUISTO BAGHERIA",
        'I.C. ':'','I.C.':'','- PA':'','-PA':'','D.D. ':'','I.C.S ':'','I.I.S. ':'','I.P.S.S.E.O.A. ':'','ITI ':'','LICEO ARTISTICO STATALE ':'',' (EX III ALBERGH)':'',
        'LICEO SCIENZE UMANE E LING. ':'','IM ':'','CPIA ':'','SCUOLA SEC. DI 1? ':'','SMS ':'','S.M.S. ':'','I.C PRINCIP. ':'','"':'',
        '" ':'','I.I.S.S. ':'',"__":"_"," _":"_"," /":"_",'/':'_',' / ':'_'," 1":"","'":"","IS MAJORANA":"MAJORANA","BORGESE-":"BORGESE_",
        " MATTARELLA -BONAGIA":"MATTARELLA BONAGIA",'SPERONE_ ':'SPERONE_',' GIOENI - ':'GIOENI_','RUSSO_ ':'RUSSO_','PEREZ-':'PEREZ_','. ':'.'
    }

    regex_dict={
        r'[ ]*$' : '',
    }

    #replace
    for key, values in replace_dict.items():
        nomescuola = nomescuola.replace(key, values)
    #regex
    for key in regex_dict:
        rx = re.compile(key)
        nomescuola = rx.sub(regex_dict[key],nomescuola)
    return nomescuola

Adesso creiamo l'ontologia relativa ai trasporti, andando a definire:
- Nome della fermata,
- Il numero del mezzo, 
- Se la tratta compiuta è un'andata o un ritorno,
- Latitudine,
- Longitudine

<i>Non tutti i file GTFS presentano un campo che indica la direzione del viaggio: se non è presente, non sarà inserito nel graph:</i>

In [13]:
# Ontologia trasporti
with open('Output_autobus.csv') as csvfile:
    reader_autobus = csv.DictReader(csvfile)
    for row in reader_autobus:
        uri_point = base_uri+"point/"+urify_fermata((row['NOMEFERMATA']))
        uri_autobus = base_uri+str(row['IDFERMATA'])
        g.add([URIRef(uri_autobus), RDF.type, skr.Autobus])
        # data properties
        g.add([URIRef(uri_autobus), skr.nomeFermata, Literal(
            row['NOMEFERMATA'], datatype=XSD.string)])
        g.add([URIRef(uri_autobus), skr.numeroAutobus, Literal(
            row['NUMEROAUTOBUS'], datatype=XSD.string)])
        try:
            g.add([URIRef(uri_autobus), skr.andata, Literal(
                row['ANDATA'], datatype=XSD.boolean)])
        except:
            pass
        g.add([URIRef(uri_point), schema.latitude,
               Literal(row['LATITUDINEFERMATA'])])
        g.add([URIRef(uri_point), schema.longitude,
               Literal(row['LONGITUDINEFERMATA'])])

        # object properties
        g.add([URIRef(uri_autobus), skr.coordinate, URIRef(uri_point)])
        g.add([URIRef(uri_point), RDF.type, schema.GeoCoordinates])

Qui andiamo a definire l'ontologia per le scuole e il plesso: il codice risulta particolarmente "impegnativo" in quanto abbiamo definito ogni singola infrastruttura come <b>Data property</b> sia per le scuole che per i plessi.

Inoltre, non è possibile separare l'if che divide l'ontologia della scuola da quella del plesso; infatti abbiamo inserito questa porzione di codice in un'unica cella.

Dettagli:

- Se il codice scuola risulta uguale al codice di riferimento dell'istituto, si tratta di una scuola "principale",

- Abbiamo anche definito i vari datatypes, ad esempio boolean o string,

- Per il collegamento tra scuola e fermata: andiamo a definire il punto della scuola ed il punto della fermata e calcoliamo la distanza utilizzando Geodesic: la distanza risulta minore di 300 metri, allora la scuola risulta vicina alla fermata,

<i>Vincenty è deprecato: usiamo Geodesic, che risulta essere più accurato</i>

- Andiamo ad effettuare sia l'interlinking del comune che del nome della scuola con DBpedia: 
 
<i>In sostanza, se il nome della scuola è ad esempio Leonardo Da Vinci, verrà effettuato l'interlinking con la risorsa "Leonardo Da Vinci" presente in DBpedia:</i>

In [14]:
#Ontologia scuola e plesso
with open('Output_finale.csv') as csvfile:
    reader_scuola = csv.DictReader(csvfile)
    for row in reader_scuola:
        uri_scuola=base_uri+str(row['CODICEISTITUTORIFERIMENTO'])
        uri_point = base_uri+"point/"+urify_indirizzo(row['INDIRIZZO'])
        uri_plesso=base_uri+str(row['CODICESCUOLA'])
        #Scuola
        if(row['CODICESCUOLA'] == row['CODICEISTITUTORIFERIMENTO']):
            g.add([URIRef(uri_scuola), RDF.type, skr.Scuola])
            #data properties
            g.add([URIRef(uri_scuola), skr.valutazione, Literal(row['RANK'], datatype=XSD.float)])
            g.add([URIRef(uri_scuola), skr.haCodiceIstitutoRiferimento, Literal(row['CODICEISTITUTORIFERIMENTO'],datatype=XSD.string)])
            g.add([URIRef(uri_scuola), skr.haDescrizioneTipologiaGradoIstruzioneScuola, Literal(row['DESCRIZIONETIPOLOGIAGRADOISTRUZIONESCUOLA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haAccessoRampe, Literal(row['ACCESSORAMPE'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haAltriAccorgimentiDisabili, Literal(row['ALTRIACCORGIMENTIDISABILI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haSpaziDidattici, Literal(row['SPAZIDIDATTICI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haSpaziAmministrativi, Literal(row['SPAZIAMMINISTRATIVI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haServiziIgieniciDisabili, Literal(row['SERVIZIIGIENICIDISABILI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haScaleNorma, Literal(row['SCALENORMA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haAscensoriDisabili, Literal(row['ASCENSOREDISABILI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haSuperamentoBarriereArchitettoniche, Literal(row['SUPERAMENTOBARRIEREARCH'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haTeleriscaldamento, Literal(row['TELERISCALDAMENTO'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haAulaMagna, Literal(row['AULAMAGNA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haCentralizzatoAria, Literal(row['CENTRALIZZATOARIA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haCentralizzatoGasolio, Literal(row['CENTRALIZZATOGASOLIO'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haCentralizzatoOlioCombustibili, Literal(row['CENTRALIZZATOOLIOCOMBUSTIBILI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haCorpiScaldantiElettriciAutonomi, Literal(row['CORPISCALDANTIELETTRICIAUTONOMI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haImpiantoRiscaldamento, Literal(row['IMPIANTORISCALDAMENTO'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haCondizionamentoVentilazione, Literal(row['CONDIZIONAMENTOVENTILAZIONE'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haPalestraPiscina, Literal(row['PALESTRAPISCINA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haRiscaldamentoAltraNatura, Literal(row['RISCALDAMENTOALTRANATURA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_scuola), skr.haSitoWeb, Literal(row['SITOWEB'],datatype=XSD.string)])
            g.add([URIRef(uri_scuola), skr.tipoScuola, Literal(row['DESCRIZIONETIPOLOGIAGRADOISTRUZIONESCUOLA'], datatype=XSD.string)])
            g.add([URIRef(uri_point), schema.latitude, Literal(row['LATITUDINE'])])
            g.add([URIRef(uri_point), schema.longitude, Literal(row['LONGITUDINE'])])
            g.add([URIRef(uri_scuola), skr.nomeScuola, Literal(row['NOMESCUOLA'])])
            #object properties
            g.add([URIRef(uri_scuola), skr.haPlesso, URIRef(uri_plesso)])
            g.add([URIRef(uri_scuola), skr.coordinate, URIRef(uri_point)])
            g.add([URIRef(uri_point), RDF.type, schema.GeoCoordinates])
            


            #interlink comune
            dbpedia_url="http://dbpedia.org/resource/"+str(row['DESCRIZIONECOMUNE']).title().replace(" ","_")
            r=requests.get(dbpedia_url)
            if(r.status_code==200):
                g.add([URIRef(uri_scuola), skr.comune, URIRef(dbpedia_url)])
            else:
                print(row['DESCRIZIONECOMUNE']+" --- "+dbpedia_url)
                g.add([URIRef(uri_scuola), skr.comune, Literal(row['DESCRIZIONECOMUNE'],datatype=XSD.string) ])
            #interlink nomescuola
            dbpedia_url="http://dbpedia.org/resource/"+str(urify_scuola(row['NOMESCUOLA'])).title().replace(" ","_")
            r=requests.get(dbpedia_url)
            if(r.status_code==200):
                g.add([URIRef(uri_scuola), skr.dedicataA, URIRef(dbpedia_url)])
            else:
                print(urify_scuola(row['NOMESCUOLA'])+" --- "+dbpedia_url)
                g.add([URIRef(uri_scuola), skr.dedicataA, Literal(None) ])
            

            punto_scuola=(row['LATITUDINE'],row['LONGITUDINE'])
            with open('Output_autobus.csv') as csvfile:
                punto_scuola=(row['LATITUDINE'],row['LONGITUDINE'])
                reader_autobus = csv.DictReader(csvfile)
                for row in reader_autobus:
                    uri_point = base_uri+"point/"+urify_fermata((row['NOMEFERMATA']))
                    latitudine_fermata = row['LATITUDINEFERMATA']
                    longitudine_fermata = row['LONGITUDINEFERMATA']
                    nome_fermata = row['NOMEFERMATA']
                    punto_fermata = (row['LATITUDINEFERMATA'], row['LONGITUDINEFERMATA'])
                    dist = geodesic(punto_scuola, punto_fermata).meters
                    if dist < 300:
                        g.add([URIRef(uri_scuola), skr.vicinoA, URIRef(uri_point)])
            

            
#Da qui in poi e' per il plesso
        else:
        #Plesso
            g.add([URIRef(uri_plesso), RDF.type, skr.Plesso])
            #data properties 
            g.add([URIRef(uri_plesso), skr.valutazione, Literal(row['RANK'],datatype=XSD.float)])
            g.add([URIRef(uri_plesso), skr.haCodiceScuola, Literal(row['CODICESCUOLA'],datatype=XSD.string)])
            g.add([URIRef(uri_plesso), skr.haDescrizioneTipologiaGradoIstruzioneScuola, Literal(row['DESCRIZIONETIPOLOGIAGRADOISTRUZIONESCUOLA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haAccessoRampe, Literal(row['ACCESSORAMPE'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haAltriAccorgimentiDisabili, Literal(row['ALTRIACCORGIMENTIDISABILI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haSpaziDidattici, Literal(row['SPAZIDIDATTICI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haSpaziAmministrativi, Literal(row['SPAZIAMMINISTRATIVI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haServiziIgieniciDisabili, Literal(row['SERVIZIIGIENICIDISABILI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haScaleNorma, Literal(row['SCALENORMA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haAscensoriDisabili, Literal(row['ASCENSOREDISABILI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haSuperamentoBarriereArchitettoniche, Literal(row['SUPERAMENTOBARRIEREARCH'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haTeleriscaldamento, Literal(row['TELERISCALDAMENTO'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haAulaMagna, Literal(row['AULAMAGNA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haCentralizzatoAria, Literal(row['CENTRALIZZATOARIA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haCentralizzatoGasolio, Literal(row['CENTRALIZZATOGASOLIO'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haCentralizzatoOlioCombustibili, Literal(row['CENTRALIZZATOOLIOCOMBUSTIBILI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haCorpiScaldantiElettriciAutonomi, Literal(row['CORPISCALDANTIELETTRICIAUTONOMI'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haImpiantoRiscaldamento, Literal(row['IMPIANTORISCALDAMENTO'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haCondizionamentoVentilazione, Literal(row['CONDIZIONAMENTOVENTILAZIONE'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haPalestraPiscina, Literal(row['PALESTRAPISCINA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haRiscaldamentoAltraNatura, Literal(row['RISCALDAMENTOALTRANATURA'],datatype=XSD.boolean)])
            g.add([URIRef(uri_plesso), skr.haSitoWeb, Literal(row['SITOWEB'],datatype=XSD.string)])
            g.add([URIRef(uri_plesso), skr.tipoScuola, Literal(row['DESCRIZIONETIPOLOGIAGRADOISTRUZIONESCUOLA'], datatype=XSD.string)])
            g.add([URIRef(uri_point), schema.latitude, Literal(row['LATITUDINE'])])
            g.add([URIRef(uri_point), schema.longitude, Literal(row['LONGITUDINE'])])
            g.add([URIRef(uri_plesso), skr.nomeScuola, Literal(row['NOMESCUOLA'])])

            #object properties
            g.add([URIRef(uri_plesso), skr.coordinate, URIRef(uri_point)])
            g.add([URIRef(uri_point), RDF.type, schema.GeoCoordinates])

            
            #interlink comune
            dbpedia_url="http://dbpedia.org/resource/"+str(row['DESCRIZIONECOMUNE']).title().replace(" ","_")
            r=requests.get(dbpedia_url)
            if(r.status_code==200):
                g.add([URIRef(uri_plesso), skr.comune, URIRef(dbpedia_url)])
            else:
                print(row['DESCRIZIONECOMUNE']+" --- "+dbpedia_url)
                g.add([URIRef(uri_plesso), skr.comune, Literal(row['DESCRIZIONECOMUNE'],datatype=XSD.string)])
            #interlink nomescuola
            dbpedia_url="http://dbpedia.org/resource/"+str(urify_scuola(row['NOMESCUOLA'])).title().replace(" ","_")
            r=requests.get(dbpedia_url)
            if(r.status_code==200):
                g.add([URIRef(uri_plesso), skr.dedicataA, URIRef(dbpedia_url)])
            else:
                print(urify_scuola(row['NOMESCUOLA'])+" --- "+dbpedia_url)
                g.add([URIRef(uri_plesso), skr.dedicataA, Literal(None)])
            

            punto_scuola=(row['LATITUDINE'],row['LONGITUDINE'])
            with open('Output_autobus.csv') as csvfile:
                reader_autobus = csv.DictReader(csvfile)
                for row in reader_autobus:
                    uri_point = base_uri+"point/"+urify_fermata((row['NOMEFERMATA']))
                    latitudine_fermata = row['LATITUDINEFERMATA']
                    longitudine_fermata = row['LONGITUDINEFERMATA']
                    nome_fermata = row['NOMEFERMATA']
                    punto_fermata = (row['LATITUDINEFERMATA'], row['LONGITUDINEFERMATA'])
                    dist = geodesic(punto_scuola, punto_fermata).meters
                    if dist < 300:
                        g.add([URIRef(uri_plesso), skr.vicinoA, URIRef(uri_point)])

ETTORE ARCULEO --- http://dbpedia.org/resource/Ettore_Arculeo
ETTORE ARCULEO --- http://dbpedia.org/resource/Ettore_Arculeo
ARISTIDE GABELLI --- http://dbpedia.org/resource/Aristide_Gabelli
ARISTIDE GABELLI --- http://dbpedia.org/resource/Aristide_Gabelli
NICOLO GARZILLI --- http://dbpedia.org/resource/Nicolo_Garzilli
NICOLO GARZILLI --- http://dbpedia.org/resource/Nicolo_Garzilli
NICOLO GARZILLI --- http://dbpedia.org/resource/Nicolo_Garzilli
NICOLO GARZILLI --- http://dbpedia.org/resource/Nicolo_Garzilli
FRANCESCO ORESTANO --- http://dbpedia.org/resource/Francesco_Orestano
FRANCESCO ORESTANO --- http://dbpedia.org/resource/Francesco_Orestano
FRANCESCO ORESTANO --- http://dbpedia.org/resource/Francesco_Orestano
FRANCESCO ORESTANO --- http://dbpedia.org/resource/Francesco_Orestano
ALESSANDRA SIRAGUSA --- http://dbpedia.org/resource/Alessandra_Siragusa
PARTANNA MONDELLO --- http://dbpedia.org/resource/Partanna_Mondello
PARTANNA MONDELLO --- http://dbpedia.org/resource/Partanna_Mondello


Salviamo il tutto in un file .ttl (Turtle).

In [15]:
g.serialize(destination='Dataset_finale.ttl', format='turtle')